# Оценка метрик ReAct Router

Этот ноутбук вычисляет метрики качества работы ReAct Router:
- Recall (Полнота)
- Precision (Точность)
- F1-score
- TP, FP, TN, FN (Confusion Matrix)

ReAct Router решает, нужен ли retriever для ответа на запрос.


In [1]:
import json
import sys
from pathlib import Path
from typing import Literal

import asyncio
import pandas as pd
import numpy as np

# Добавляем путь к проекту
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / "src"))

print(f"📁 Корневая директория проекта: {project_root}")


📁 Корневая директория проекта: /srv/nlp1/T-bank_NLP_-1


In [2]:
from tplexity.generation.generation_service import GenerationService
from tplexity.generation.config import settings

print("✅ Импорты выполнены успешно")


✅ Импорты выполнены успешно


## Загрузка данных


In [3]:
# Путь к датасету
dataset_path = project_root / "data" / "data_18.11.25" / "react_router_dataset.json"

with open(dataset_path, "r", encoding="utf-8") as f:
    dataset = json.load(f)

print(f"📊 Загружено записей: {len(dataset)}")
print(f"\nПример записи:")
print(json.dumps(dataset[0], ensure_ascii=False, indent=2))


📊 Загружено записей: 2895

Пример записи:
{
  "query": "Каковы финансовые результаты «Озона» и «Группы Позитив» за 9 месяцев 2025 года?",
  "doc_id": "0065e9a4-858b-4976-877f-d9f1d463c295",
  "document_text": "Дата публикации: 10 ноября 2025\n\n**⏰**** Доброе утро: 10 ноября**\n\n🔹 «Озон» и «Группа Позитив» опубликуют финансовые результаты за 9 месяцев 2025 года. \n\n🔹 Сенат США набрал необходимое число голосов за сделку по возобновлению финансирования правительства США. Это поддерживает оптимизм на финансовых рынках. Азиатские фондовые индексы, цены на сырьё и криптовалюты растут. Золото торгуется выше $4000.\n\n🔹 Инфляция в Китае в октябре выросла на 0,2% г/г — выше прогнозов, это максимальное значение за 9 месяцев. Индекс цен производителей  упал на 2, 1% г/г, снижение продолжается уже 2 года. \n\n**☕**** Мысли с утра \n**\nЗавтра на бирже стартуют торги акциями «Озон» после переезда в Россию. \n\nОбъявление первых в истории дивидендов компании, в нашем понимании, станет одним из фа

## Инициализация GenerationService


In [4]:
# Инициализируем GenerationService
generation_service = GenerationService(
    llm_provider=settings.llm_provider,
    retriever_url=settings.retriever_api_url,
)

print(f"✅ GenerationService инициализирован")
print(f"   LLM Provider: {generation_service.llm_provider}")


✅ GenerationService инициализирован
   LLM Provider: qwen


## Функция для получения предсказания Router


In [11]:
async def get_router_prediction(query: str, llm_provider: str | None = None) -> Literal["YES", "NO"]:
    """
    Получает предсказание ReAct Router для запроса.
    
    Args:
        query: Запрос пользователя
        llm_provider: Провайдер LLM для принятия решения
    
    Returns:
        "YES" если нужен retriever, "NO" если не нужен
    """
    use_retriever = await generation_service._should_use_retriever(
        query=query,
        session_id=None,  # Без истории диалога
        llm_provider=llm_provider,
    )
    return "YES" if use_retriever else "NO"


In [14]:
result = await get_router_prediction("что такое мосбиржа?")

In [15]:
result

'YES'

## Оценка на датасете


In [6]:
async def evaluate_dataset(dataset: list[dict], batch_size: int = 10) -> list[dict]:
    """
    Оценивает датасет и возвращает результаты.
    
    Args:
        dataset: Список записей датасета
        batch_size: Размер батча для обработки
    
    Returns:
        Список результатов с предсказаниями
    """
    results = []
    
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i + batch_size]
        
        # Обрабатываем батч параллельно
        tasks = []
        for record in batch:
            query = record["query"]
            task = get_router_prediction(query)
            tasks.append(task)
        
        predictions = await asyncio.gather(*tasks)
        
        # Сохраняем результаты
        for record, prediction in zip(batch, predictions):
            expected = record.get("expected_router_decision", "YES" if record.get("requires_rag", True) else "NO")
            requires_rag = record.get("requires_rag", expected == "YES")
            
            results.append({
                "query": record["query"],
                "expected": expected,
                "requires_rag": requires_rag,
                "predicted": prediction,
                "correct": prediction == expected,
            })
        
        if (i + batch_size) % 100 == 0 or i + batch_size >= len(dataset):
            print(f"✅ Обработано: {min(i + batch_size, len(dataset))} / {len(dataset)}")
    
    return results


In [7]:
# Запускаем оценку
print(f"🚀 Начинаем оценку на {len(dataset)} запросах...")
results = await evaluate_dataset(dataset, batch_size=10)
print(f"\n✅ Оценка завершена!")


🚀 Начинаем оценку на 2895 запросах...
✅ Обработано: 100 / 2895
✅ Обработано: 200 / 2895
✅ Обработано: 300 / 2895
✅ Обработано: 400 / 2895
✅ Обработано: 500 / 2895
✅ Обработано: 600 / 2895
✅ Обработано: 700 / 2895
✅ Обработано: 800 / 2895
✅ Обработано: 900 / 2895
✅ Обработано: 1000 / 2895
✅ Обработано: 1100 / 2895
✅ Обработано: 1200 / 2895
✅ Обработано: 1300 / 2895
✅ Обработано: 1400 / 2895
✅ Обработано: 1500 / 2895
✅ Обработано: 1600 / 2895
✅ Обработано: 1700 / 2895
✅ Обработано: 1800 / 2895
✅ Обработано: 1900 / 2895
✅ Обработано: 2000 / 2895
✅ Обработано: 2100 / 2895
✅ Обработано: 2200 / 2895
✅ Обработано: 2300 / 2895
✅ Обработано: 2400 / 2895
✅ Обработано: 2500 / 2895
✅ Обработано: 2600 / 2895
✅ Обработано: 2700 / 2895
✅ Обработано: 2800 / 2895
✅ Обработано: 2895 / 2895

✅ Оценка завершена!


## Вычисление метрик


In [8]:
def calculate_metrics(results: list[dict]) -> dict:
    """
    Вычисляет метрики на основе результатов.
    
    Args:
        results: Список результатов с предсказаниями
    
    Returns:
        Словарь с метриками
    """
    # Подсчитываем TP, FP, TN, FN
    # Для ReAct Router:
    # - YES означает "использовать retriever" (positive class)
    # - NO означает "не использовать retriever" (negative class)
    
    tp = 0  # True Positive: predicted=YES, actual=YES (requires_rag=True)
    fp = 0  # False Positive: predicted=YES, actual=NO (requires_rag=False)
    tn = 0  # True Negative: predicted=NO, actual=NO (requires_rag=False)
    fn = 0  # False Negative: predicted=NO, actual=YES (requires_rag=True)
    
    for result in results:
        predicted = result["predicted"]
        requires_rag = result["requires_rag"]
        
        if predicted == "YES" and requires_rag:
            tp += 1
        elif predicted == "YES" and not requires_rag:
            fp += 1
        elif predicted == "NO" and not requires_rag:
            tn += 1
        elif predicted == "NO" and requires_rag:
            fn += 1
    
    # Вычисляем метрики
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0.0
    
    return {
        "tp": tp,
        "fp": fp,
        "tn": tn,
        "fn": fn,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy,
        "total": len(results),
    }


In [9]:
# Вычисляем метрики
metrics = calculate_metrics(results)

print("📊 Результаты оценки ReAct Router:")
print("=" * 60)
print(f"Всего запросов: {metrics['total']}")
print(f"\nConfusion Matrix:")
print(f"  TP (True Positive):  {metrics['tp']:4d}  |  Предсказано YES, фактически YES")
print(f"  FP (False Positive): {metrics['fp']:4d}  |  Предсказано YES, фактически NO")
print(f"  TN (True Negative):  {metrics['tn']:4d}  |  Предсказано NO,  фактически NO")
print(f"  FN (False Negative): {metrics['fn']:4d}  |  Предсказано NO,  фактически YES")
print(f"\nМетрики:")
print(f"  Precision: {metrics['precision']:.4f}  (TP / (TP + FP))")
print(f"  Recall:    {metrics['recall']:.4f}  (TP / (TP + FN))")
print(f"  F1-score:  {metrics['f1']:.4f}")
print(f"  Accuracy:  {metrics['accuracy']:.4f}  ((TP + TN) / Total)")
print("=" * 60)


📊 Результаты оценки ReAct Router:
Всего запросов: 2895

Confusion Matrix:
  TP (True Positive):  1442  |  Предсказано YES, фактически YES
  FP (False Positive): 1446  |  Предсказано YES, фактически NO
  TN (True Negative):     0  |  Предсказано NO,  фактически NO
  FN (False Negative):    7  |  Предсказано NO,  фактически YES

Метрики:
  Precision: 0.4993  (TP / (TP + FP))
  Recall:    0.9952  (TP / (TP + FN))
  F1-score:  0.6650
  Accuracy:  0.4981  ((TP + TN) / Total)


In [10]:
# Создаем DataFrame для анализа
df = pd.DataFrame(results)

# Ошибки типа FP (False Positive)
fp_errors = df[(df["predicted"] == "YES") & (df["requires_rag"] == False)]
print(f"\n❌ False Positive (FP) ошибки: {len(fp_errors)}")
print("   Router решил использовать retriever, но это было не нужно")
if len(fp_errors) > 0:
    print("\nПримеры FP ошибок:")
    for idx, row in fp_errors.head(5).iterrows():
        print(f"\n  {idx + 1}. {row['query'][:100]}...")

# Ошибки типа FN (False Negative)
fn_errors = df[(df["predicted"] == "NO") & (df["requires_rag"] == True)]
print(f"\n❌ False Negative (FN) ошибки: {len(fn_errors)}")
print("   Router решил НЕ использовать retriever, но он был нужен")
if len(fn_errors) > 0:
    print("\nПримеры FN ошибок:")
    for idx, row in fn_errors.head(5).iterrows():
        print(f"\n  {idx + 1}. {row['query'][:100]}...")



❌ False Positive (FP) ошибки: 1446
   Router решил использовать retriever, но это было не нужно

Примеры FP ошибок:

  1450. Что такое дивидендная политика компании?...

  1451. Какие факторы влияют на стоимость акций на фондовом рынке?...

  1452. Что такое рыночная капитализация и как она рассчитывается?...

  1453. Что такое рыночная капитализация и как она рассчитывается?...

  1454. Какие факторы влияют на стоимость облигаций на фондовом рынке?...

❌ False Negative (FN) ошибки: 7
   Router решил НЕ использовать retriever, но он был нужен

Примеры FN ошибок:

  21. Как подписаться на канал «Сам ты инвестор!»?...

  348. Какой канал можно использовать для задания вопросов в прямом эфире?...

  495. Какое время было при генерации арбуза?...

  609. Где можно посмотреть трансляцию пленарной дискуссии конференции «Фокус на клиента»?...

  873. Какое время было на документе?...


## Сохранение результатов
